# Open discovery model

## Motivation

The main idea behind Swanson's work is that the two originally unconnected parts of the literature (C and A) can establish latent connections (through intermediate terms in B). These two pieces of literature have one or more b-terms in common, creating an indirect connection. Once these bridges are identified and validated, new knowledge can be generated.

Swanson founded his discovery on the so-called ABC model, which is now widely regarded as a typical paradigm for LBD. The ABC model contains three concepts: the start concept ($c$), the intermediate concept ($b$), and the target concept ($a$). The LBD process begins with retrieving $c-b$ and $b-a$ relationships. Next, it combines associations with the same intermediates. Finally, it retrieves a list of $a-c$ relationships. If there is no prior mention of a particular $a-c$ connection, a hypothesis of a potential novel relationship between $a$ and $c$ concepts is conceived.

<img src="img/open_discovery.png" alt="Open discovery model" width="300px"/>

In Swanson's scenario from 1988, literature domain $C$ refers to migraine (i.e., a set of papers mentioning migraine), whereas domain $A$ represents the fish oil literature (set of papers mentioning fish oil).

Shortly after his seminal paper on ``undiscovered public knowledge'', \citet{swanson1988migraine}, following the same discovery procedure, reported observations on the role of magnesium in migraine disorder. Magnesium insufficiency may exacerbate migraine due to complications involving stress (in connection with Type A personality), spreading cortical depression, epilepsy, platelet aggregation, serotonin, substance P, inflammation, vasoconstriction, prostaglandin formation, and hypoxia. Conversely, the ability of magnesium to block calcium channels may help to prevent migraine episodes.

Using the Entrez query
\begin{minted}[breaklines]{text}
migraine[TIAB] AND 1900/01/31:1987/12/31 [PDAT] AND medline [SB] AND english [LA]
\end{minted}
we retrieved 3,058 bibliographic records from PubMed (as of May 17, 2024). The search strategy included four components. First, we manually selected the term "magnesium" to retrieve all relevant papers containing a source term in the title and/or abstract fields (\texttt{TIAB}). Second, we applied a date range constraint to match the same period as in Swanson's work (\texttt{PDAT}). Third, only high-quality articles published in the MEDLINE subset of PubMed were retrieved (\texttt{SB}). The fourth component included articles written only in English (\texttt{LA}).

To map free text from the collected titles and abstracts to the UMLS concepts, we used the MetaMap tool.

To reduce the search space during the LBD process, we can filter the UMLS-mapped concepts using semantic types. Semantic filters are query dependent. For instance, in the stage of selecting bridging $b$-concepts, we might select only those concepts that have a functional semantic type, such as ``Biologic Function'', ``Cell Function'', and ``Phenomenon or Process''.

The open discovery setting is illustrated in Figure~\ref{fig-OpenDiscovery}. Open discovery is in essence the approach employed for generating new hypotheses. Let us describe it in terms of Swanson's $ABC$ \emph{model} \citep{swanson1986undiscovered}. Let the starting point of research be term $c$, let $C$ be the literature about $c$, and let us assume that domain $A$ consists of the literature that might contain some---yet unknown---connections to $C$. The goal is to find $A$. This is done 
\begin{quote}
via some intermediate literature ($B$) toward an unknown destination $A$\ldots  Success depends entirely on the knowledge and ingenuity of the searcher\ldots \citep{swanson1990medical}
\end{quote}

!pip install scikit-learn
!pip install tabulate

# Load required libraries for NLP and data analysis
import gzip
import numpy as np
import pandas as pd
import re
# import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

# Read migraine Medline data with MeSH headings
df_3 = pd.read_csv('./mig_mg/pmid_dp_ti_mh_20241124.psv.gz', sep='|', names=['pmid','dp','ti','mh_lst'])
df_3.head()

df_4 = pd.DataFrame(df_3['mh_lst'].str.split(';').tolist(), index=df_3.pmid).stack().reset_index([0, 'pmid'])
df_4.columns = ['pmid', 'mh']
df_4

###########################
# Parse a Descriptor Record

import os
# import pandas as pd
from collections import defaultdict
from itertools import groupby, filterfalse
from collections import Counter


# https://www.nlm.nih.gov/research/umls/knowledge_sources/semantic_network/SemGroups.txt
def get_semantic_types(category=None):
    # read in semantic types
    st_df = pd.read_csv("https://semanticnetwork.nlm.nih.gov/download/SemGroups.txt", delimiter="|",
                        names=["x0", "x1", "x2", "x3"])
    st_df = pd.read_csv("https://www.nlm.nih.gov/research/umls/knowledge_sources/semantic_network/SemGroups.txt", delimiter="|",
                        names=["x0", "x1", "x2", "x3"])
    #st_d = dict(zip(st_df['x2'], st_df['x3']))
    # cat_df = st_df.query("x1 == @category")[['x2', 'x3']]
    # semantic_types = set(cat_df['x2'])
    # disorders = {'T037', 'T049', 'T048', 'T050', 'T184', 'T019', 'T190', 'T020', 'T033', 'T046', 'T191', 'T047'}
    semantic_types = st_df
    return semantic_types

def parse_descriptor_records(mesh_desc_path, semantic_types = None):
    """
    semantic_types is optional

    :param mesh_desc_path:
    :return:
    """
    #TODO: only care about disease related attributes. !!!
    # for example, ignoring RN CAS REGISTRY/EC NUMBER/UNII CODE  !!!

    attributes = {'MH': "term",
                  'MN': "tree",
                  'FX': "see_also",
                  'ST': "semantic_type",  # see: https://semanticnetwork.nlm.nih.gov/download/SemGroups.txt
                  'MS': "note",
                  'MR': "last_updated",
                  'DC': "descriptor_class",
                  'UI': "_id",
                  'RECTYPE': "record_type",
                  'synonyms': "synonyms"}  # added by me from PRINT ENTRY & ENTRY

    # TODO: parse PRINT ENTRY and ENTRY completely
    # "'D-2-hydroxyglutaric aciduria|T047|EQV|OMIM (2013)|ORD (2010)|090615|abdeef'"


    # read in the mesh data
    with open(mesh_desc_path) as f:
        mesh_desc = [x.strip() for x in f.readlines()]

    # which attributes can have multiple values?
    gb = filterfalse(lambda x: x[0], groupby(mesh_desc, lambda x: x == "*NEWRECORD"))
    ds = []
    for gb_record in gb:
        record = list(gb_record[1])
        d = dict(Counter([line.split("=", 1)[0].strip() for line in record if "=" in line]))
        ds.append(d)
    df = pd.DataFrame(ds).fillna(0)
    list_attribs = set(df.columns[df.max() > 1])
    # list_attribs = {'EC', 'ENTRY', 'FX', 'MH_TH', 'MN', 'PA', 'PI', 'PRINT ENTRY', 'RR', 'ST'}

    # split into records
    gb = filterfalse(lambda x: x[0], groupby(mesh_desc, lambda x: x == "*NEWRECORD"))

    mesh_terms = dict()
    for gb_record in gb:
        record = list(gb_record[1])
        d = defaultdict(list)
        for line in record:
            if "=" not in line:
                continue
            key = line.split("=", 1)[0].strip()
            value = line.split("=", 1)[1].strip()
            if key not in attributes and key not in list_attribs:
                d[key] = value
            elif key in list_attribs and key in attributes:
                d[attributes[key]].append(value)
            elif key in attributes and key not in {"PRINT ENTRY", "ENTRY"}:
                d[attributes[key]] = value
            elif key in {"PRINT ENTRY", "ENTRY"}:
                d['synonyms'].append(value.split("|", 1)[0])

        if semantic_types and not (set(d['semantic_type']) & semantic_types):
            continue
        mesh_terms[d['_id']] = dict(d)

    return mesh_terms
###########################

sty_df = get_semantic_types()

sty_df.head()
# print(sty_df.to_markdown())
# X,X,UI,STY

# For convenience, we have included a parsable list of Semantic Types and their abbreviations from the UMLS via the link below. The format of the file is "Abbreviation|Type Unique Identifier (TUI)|Full Semantic Type Name". For additional information on Semantic Types, please refer to The UMLS Semantic Network website, the UMLS Semantic Network Fact Sheet, and the Current Semantic Types website .

#  Semantic Type Mappings (3.6 kb)

# Also, we have included a parsable list of Semantic Groups and their mappings to the Semantic Types via the link below. The format of the file is "Semantic Group Abbrev|Semantic Group Name|TUI|Full Semantic Type Name". For additional information on Semantic Groups, please refer to the Semantic Groups web site.




#####
# sty_filt

SEM_FILT_5 = {'biof': 'Biologic Function',
            'blor': 'Body Location or Region',
            'celf': 'Cell Function',
            'phpr': 'Phenomenon or Process',
            'phsf': 'Physiologic Function',
           }

SEM_FILT_9 = {'biof': 'Biologic Function',
            'celf': 'Cell Function',
            'fndg': 'Finding',
            'moft': 'Molecular Function',
            'ortf': 'Organ or Tissue Function',
            'orgf': 'Organism Function',
            'patf': 'Pathologic Function',
            'phpr': 'Phenomenon or Process',
            'phsf': 'Physiologic Function',
           }

SEM_FILT_10 = {
    'aapp': 'Amino Acid, Peptide, or Protein',
    'biof': 'Biologic Function',
            'celf': 'Cell Function',
            'fndg': 'Finding',
            'moft': 'Molecular Function',
            'ortf': 'Organ or Tissue Function',
            'orgf': 'Organism Function',
            'patf': 'Pathologic Function',
            'phpr': 'Phenomenon or Process',
            'phsf': 'Physiologic Function',
           }

sty_filt_lst = [*SEM_FILT_10.values()]

sty_filt_tui = sty_df.query('x3 in @sty_filt_lst')['x2'].to_list()
#####

#####
DATA_DIR = './mesh'
mesh_desc_path = os.path.join(DATA_DIR, "d2024.bin")
# mesh_supp_path = os.path.join(DATA_DIR, "c2017.bin")

mesh_terms = parse_descriptor_records(mesh_desc_path, semantic_types=None)
#####

#####
# mesh_terms
# mesh_terms['D000001']['term']
ui2term_dct = {k: v['term'] for k, v in mesh_terms.items()}
ui2dc_dct = {k: v['descriptor_class'] for k, v in mesh_terms.items()}
#####

#####
ui2tree_dct = {}

for k, v in mesh_terms.items():
    if 'tree' in v:
        ui2tree_dct[k] = v['tree']
    else:
        ui2tree_dct[k] = None

# mesh_terms['D005260']
#####

#####
df_5 = pd.DataFrame.from_dict(ui2term_dct.items())
df_5.columns = ['ui', 'term']
df_5
#####

#####
df_5['document_class'] = df_5['ui'].map(ui2dc_dct)
df_5['tree'] = df_5['ui'].map(ui2tree_dct)
#####

#####
df_6 = df_4.merge(right=df_5, how='left', left_on='mh', right_on='term')
df_6 = df_6.filter(['pmid', 'ui', 'term', 'document_class', 'tree'])
df_6
#####

# D008787	Metoclopramide
mesh_terms['D008787']
mesh_terms['D008787']['semantic_type']

res = []

for ui in df_6['ui']:
    ui_sty_lst = mesh_terms[ui]['semantic_type']
    if (set(ui_sty_lst) & set(sty_filt_tui)):
        res.append(ui)

ui_sty_lst
sty_filt_tui
set(['T001', 'T046']) & set(sty_filt_tui)

print(f'{len(res)}')
print(f'{df_6.shape}')
df_6.head()

df_6
df_7 = df_6.query('ui in @res')
print(f'{df_7.shape}')
# print(df_7.sort_values('term').to_markdown())

print(f'No. of PMIDs: {df_6['pmid'].nunique()}')
print(f'No. of MeSH terms: {df_6['term'].size}')

print(f'No. of unique UIs: {df_7['ui'].nunique()}')
print(f'No. of unique MeSH terms: {df_7['term'].nunique()}')

l2l = df_7.groupby('pmid')['term'].apply(list).to_list()

# From l2l_sel to TFIDF
tf = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, min_df=3)
tf_fit = tf.fit_transform(l2l)
wrd_lst = tf.get_feature_names_out()

score_lst = np.array(tf_fit.sum(axis=0)).reshape(-1).tolist()
wrd2score = dict(zip(wrd_lst, score_lst))

df_tf_sel = (pd.DataFrame()
             .from_dict(wrd2score, orient='index')
             .reset_index()
             .set_axis(['name', 'score'], axis=1)
             .sort_values(by='score', ascending=False)
            )

print(df_tf_sel.head(n = 10).to_markdown())
# df_tf_sel

B -> A: Vasoconstriction
df_81 = pd.read_csv('./mig_mg/pmid_dp_ti_mh_vasoconstriction_20241125.psv.gz', sep='|', names=['pmid','dp','ti','mh_lst'])
df_82 = pd.DataFrame(df_81['mh_lst'].str.split(';').tolist(), index=df_81.pmid).stack().reset_index([0, 'pmid'])
df_82.columns = ['pmid', 'mh']

print(f'No. of PubMed records: {df_81.shape}')
df_82.head()

df_83 = df_82.merge(right=df_5, how='left', left_on='mh', right_on='term')
df_83 = df_83.filter(['pmid', 'ui', 'term'])
df_83.head()

res = []

for ui in df_83['ui']:
    ui_sty_lst = mesh_terms[ui]['semantic_type']
    if (set(ui_sty_lst) & set(['T127', 'T196'])):
        res.append(ui)

# T127 | Vitamin
# T196 | Element, Ion, or Isotope

df_84 = df_83.query('ui in @res')
# print(df_84.sort_values('term').to_markdown())

l2l = df_84.groupby('pmid')['term'].apply(list).to_list()
# From l2l_sel to TFIDF
tf = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, min_df=3)
tf_fit = tf.fit_transform(l2l)
wrd_lst = tf.get_feature_names_out()

score_lst = np.array(tf_fit.sum(axis=0)).reshape(-1).tolist()
wrd2score = dict(zip(wrd_lst, score_lst))

df_tf_sel_1 = (pd.DataFrame()
             .from_dict(wrd2score, orient='index')
             .reset_index()
             .set_axis(['name', 'score'], axis=1)
             .sort_values(by='score', ascending=False)
            )

print(f'No. of unique UIs: {df_84['ui'].nunique()}')
print(f'No. of unique MeSH terms: {df_84['term'].nunique()}')

df_tf_sel_1.head(n = 10)

B -> A: Platelet Aggregation¶

In [ ]:
# Load dataset for the first domain (migraine literature)
# The dataset is stored as pipe-separated value (PSV) compressed files.
# Only PMID and MeSH Headings columns are used for subsequent processing.
df_3 = pd.read_csv('./pmid_dp_ti_mh_20241124.psv.gz', sep='|', names=['pmid','dp','ti','mh_lst'])
df_3.head()

In [ ]:
df_4 = pd.DataFrame(df_3['mh_lst'].str.split(';').tolist(), index=df_3.pmid).stack().reset_index([0, 'pmid'])
df_4.columns = ['pmid', 'mh']
df_4

In [ ]:
# df_5 = pd.read_table('../swanson/mrconso_eng_msh_mh.psv.gz', sep='|', header=None, usecols=[*range(0, 18)], nrows=10)

# https://github.com/fagan2888/mesh-parser/blob/8bc3f82a6ef600018350222ca3c31007ab238e3a/parser.py#L9

# Parse a Descriptor Record

import os
import pandas as pd
from collections import defaultdict
from itertools import groupby, filterfalse
from collections import Counter


# https://www.nlm.nih.gov/research/umls/knowledge_sources/semantic_network/SemGroups.txt
def get_semantic_types(category=None):
    # read in semantic types
    # st_df = pd.read_csv("https://semanticnetwork.nlm.nih.gov/download/SemGroups.txt", delimiter="|",
    #                     names=["x0", "x1", "x2", "x3"])
    st_df = pd.read_csv("https://www.nlm.nih.gov/research/umls/knowledge_sources/semantic_network/SemGroups.txt", delimiter="|",
                        names=["x0", "x1", "x2", "x3"])
    #st_d = dict(zip(st_df['x2'], st_df['x3']))
    # cat_df = st_df.query("x1 == @category")[['x2', 'x3']]
    # semantic_types = set(cat_df['x2'])
    # disorders = {'T037', 'T049', 'T048', 'T050', 'T184', 'T019', 'T190', 'T020', 'T033', 'T046', 'T191', 'T047'}
    semantic_types = st_df
    return semantic_types

def parse_descriptor_records(mesh_desc_path, semantic_types = None):
    """
    semantic_types is optional

    :param mesh_desc_path:
    :return:
    """
    #TODO: only care about disease related attributes. !!!
    # for example, ignoring RN CAS REGISTRY/EC NUMBER/UNII CODE  !!!

    attributes = {'MH': "term",
                  'MN': "tree",
                  'FX': "see_also",
                  'ST': "semantic_type",  # see: https://semanticnetwork.nlm.nih.gov/download/SemGroups.txt
                  'MS': "note",
                  'MR': "last_updated",
                  'DC': "descriptor_class",
                  'UI': "_id",
                  'RECTYPE': "record_type",
                  'synonyms': "synonyms"}  # added by me from PRINT ENTRY & ENTRY

    # TODO: parse PRINT ENTRY and ENTRY completely
    # "'D-2-hydroxyglutaric aciduria|T047|EQV|OMIM (2013)|ORD (2010)|090615|abdeef'"


    # read in the mesh data
    with open(mesh_desc_path) as f:
        mesh_desc = [x.strip() for x in f.readlines()]

    # which attributes can have multiple values?
    gb = filterfalse(lambda x: x[0], groupby(mesh_desc, lambda x: x == "*NEWRECORD"))
    ds = []
    for gb_record in gb:
        record = list(gb_record[1])
        d = dict(Counter([line.split("=", 1)[0].strip() for line in record if "=" in line]))
        ds.append(d)
    df = pd.DataFrame(ds).fillna(0)
    list_attribs = set(df.columns[df.max() > 1])
    # list_attribs = {'EC', 'ENTRY', 'FX', 'MH_TH', 'MN', 'PA', 'PI', 'PRINT ENTRY', 'RR', 'ST'}

    # split into records
    gb = filterfalse(lambda x: x[0], groupby(mesh_desc, lambda x: x == "*NEWRECORD"))

    mesh_terms = dict()
    for gb_record in gb:
        record = list(gb_record[1])
        d = defaultdict(list)
        for line in record:
            if "=" not in line:
                continue
            key = line.split("=", 1)[0].strip()
            value = line.split("=", 1)[1].strip()
            if key not in attributes and key not in list_attribs:
                d[key] = value
            elif key in list_attribs and key in attributes:
                d[attributes[key]].append(value)
            elif key in attributes and key not in {"PRINT ENTRY", "ENTRY"}:
                d[attributes[key]] = value
            elif key in {"PRINT ENTRY", "ENTRY"}:
                d['synonyms'].append(value.split("|", 1)[0])

        if semantic_types and not (set(d['semantic_type']) & semantic_types):
            continue
        mesh_terms[d['_id']] = dict(d)

    return mesh_terms


In [ ]:
sty_df = get_semantic_types()

In [ ]:
sty_df
# print(sty_df.to_markdown())

In [ ]:
sty_filt_lst = [*SEM_FILT_10.values()]

sty_filt_tui = sty_df.query('x3 in @sty_filt_lst')['x2'].to_list()

In [ ]:
DATA_DIR = '../mesh'
mesh_desc_path = os.path.join(DATA_DIR, "d2024.bin")
# mesh_supp_path = os.path.join(DATA_DIR, "c2017.bin")

mesh_terms = parse_descriptor_records(mesh_desc_path, semantic_types=None)

In [ ]:
# mesh_terms
# mesh_terms['D000001']['term']
ui2term_dct = {k: v['term'] for k, v in mesh_terms.items()}
ui2dc_dct = {k: v['descriptor_class'] for k, v in mesh_terms.items()}

In [ ]:
ui2tree_dct = {}

for k, v in mesh_terms.items():
    if 'tree' in v:
        ui2tree_dct[k] = v['tree']
    else:
        ui2tree_dct[k] = None

# mesh_terms['D005260']

In [ ]:
df_5 = pd.DataFrame.from_dict(ui2term_dct.items())
df_5.columns = ['ui', 'term']
df_5

In [ ]:
df_5['document_class'] = df_5['ui'].map(ui2dc_dct)
df_5['tree'] = df_5['ui'].map(ui2tree_dct)

In [ ]:
df_5

In [ ]:
df_6 = df_4.merge(right=df_5, how='left', left_on='mh', right_on='term')
df_6 = df_6.filter(['pmid', 'ui', 'term', 'document_class', 'tree'])
df_6

In [ ]:
# D008787	Metoclopramide
mesh_terms['D008787']
mesh_terms['D008787']['semantic_type']

In [ ]:
res = []

for ui in df_6['ui']:
    ui_sty_lst = mesh_terms[ui]['semantic_type']
    if (set(ui_sty_lst) & set(sty_filt_tui)):
        res.append(ui)

In [ ]:
ui_sty_lst

In [ ]:
sty_filt_tui
set(['T001', 'T046']) & set(sty_filt_tui)

In [ ]:
len(res)
df_6.shape
df_6

In [ ]:
df_6
df_7 = df_6.query('ui in @res')
print(f'{df_7.shape}')
# print(df_7.sort_values('term').to_markdown())

In [ ]:
print(f'No. of PMIDs: {df_6['pmid'].nunique()}')
print(f'No. of MeSH terms: {df_6['term'].size}')

print(f'No. of unique UIs: {df_7['ui'].nunique()}')
print(f'No. of unique MeSH terms: {df_7['term'].nunique()}')

In [ ]:
l2l = df_7.groupby('pmid')['term'].apply(list).to_list()

In [ ]:
# From l2l_sel to TFIDF
tf = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, min_df=3)
tf_fit = tf.fit_transform(l2l)
wrd_lst = tf.get_feature_names_out()

score_lst = np.array(tf_fit.sum(axis=0)).reshape(-1).tolist()
wrd2score = dict(zip(wrd_lst, score_lst))

df_tf_sel = (pd.DataFrame()
             .from_dict(wrd2score, orient='index')
             .reset_index()
             .set_axis(['name', 'score'], axis=1)
             .sort_values(by='score', ascending=False)
            )

In [ ]:
print(df_tf_sel.to_markdown())
# df_tf_sel

## B -> A :: Vasoconstriction

In [ ]:
df_81 = pd.read_csv('./pmid_dp_ti_mh_vasoconstriction_20241125.psv.gz', sep='|', names=['pmid','dp','ti','mh_lst'])
df_82 = pd.DataFrame(df_81['mh_lst'].str.split(';').tolist(), index=df_81.pmid).stack().reset_index([0, 'pmid'])
df_82.columns = ['pmid', 'mh']

In [ ]:
print(f'No. of PubMed records: {df_81.shape}')
df_82

In [ ]:
df_83 = df_82.merge(right=df_5, how='left', left_on='mh', right_on='term')
df_83 = df_83.filter(['pmid', 'ui', 'term'])
df_83

In [ ]:
res = []

for ui in df_83['ui']:
    ui_sty_lst = mesh_terms[ui]['semantic_type']
    if (set(ui_sty_lst) & set(['T127', 'T196'])):
        res.append(ui)

# T127 | Vitamin
# T196 | Element, Ion, or Isotope

In [ ]:
df_84 = df_83.query('ui in @res')
# print(df_84.sort_values('term').to_markdown())

l2l = df_84.groupby('pmid')['term'].apply(list).to_list()
# From l2l_sel to TFIDF
tf = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, min_df=3)
tf_fit = tf.fit_transform(l2l)
wrd_lst = tf.get_feature_names_out()

score_lst = np.array(tf_fit.sum(axis=0)).reshape(-1).tolist()
wrd2score = dict(zip(wrd_lst, score_lst))

df_tf_sel_1 = (pd.DataFrame()
             .from_dict(wrd2score, orient='index')
             .reset_index()
             .set_axis(['name', 'score'], axis=1)
             .sort_values(by='score', ascending=False)
            )

In [ ]:
print(f'No. of unique UIs: {df_84['ui'].nunique()}')
print(f'No. of unique MeSH terms: {df_84['term'].nunique()}')

In [ ]:
df_tf_sel_1

## B -> A :: Platelet Aggregation

In [ ]:
df_91 = pd.read_csv('./pmid_dp_ti_mh_platelet_20241125.psv.gz', sep='|', names=['pmid','dp','ti','mh_lst'])
df_92 = pd.DataFrame(df_91['mh_lst'].str.split(';').tolist(), index=df_91.pmid).stack().reset_index([0, 'pmid'])
df_92.columns = ['pmid', 'mh']

df_93 = df_92.merge(right=df_5, how='left', left_on='mh', right_on='term')
df_93 = df_93.filter(['pmid', 'ui', 'term'])

res = []

for ui in df_93['ui']:
    ui_sty_lst = mesh_terms[ui]['semantic_type']
    if (set(ui_sty_lst) & set(['T127', 'T196'])):
        res.append(ui)

df_94 = df_93.query('ui in @res')

l2l = df_94.groupby('pmid')['term'].apply(list).to_list()
# From l2l_sel to TFIDF
tf = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, min_df=3)
tf_fit = tf.fit_transform(l2l)
wrd_lst = tf.get_feature_names_out()

score_lst = np.array(tf_fit.sum(axis=0)).reshape(-1).tolist()
wrd2score = dict(zip(wrd_lst, score_lst))

df_tf_sel_2 = (pd.DataFrame()
             .from_dict(wrd2score, orient='index')
             .reset_index()
             .set_axis(['name', 'score'], axis=1)
             .sort_values(by='score', ascending=False)
            )

In [ ]:
print(f'No. of PubMed records: {df_91.shape}')
print(f'No. of unique UIs: {df_94['ui'].nunique()}')
print(f'No. of unique MeSH terms: {df_94['term'].nunique()}')

# No. of PubMed records: (6273, 4)
# No. of unique UIs: 73
# No. of unique MeSH terms: 73

## B -> A :: Spreading Cortical Depression

In [ ]:
df_101 = pd.read_csv('./pmid_dp_ti_mh_cortical_20241125.psv.gz', sep='|', names=['pmid','dp','ti','mh_lst'])
df_102 = pd.DataFrame(df_101['mh_lst'].str.split(';').tolist(), index=df_101.pmid).stack().reset_index([0, 'pmid'])
df_102.columns = ['pmid', 'mh']

df_103 = df_102.merge(right=df_5, how='left', left_on='mh', right_on='term')
df_103 = df_103.filter(['pmid', 'ui', 'term'])

res = []

for ui in df_103['ui']:
    ui_sty_lst = mesh_terms[ui]['semantic_type']
    if (set(ui_sty_lst) & set(['T127', 'T196'])):
        res.append(ui)

df_104 = df_103.query('ui in @res')

l2l = df_104.groupby('pmid')['term'].apply(list).to_list()
# From l2l_sel to TFIDF
tf = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, min_df=3)
tf_fit = tf.fit_transform(l2l)
wrd_lst = tf.get_feature_names_out()

score_lst = np.array(tf_fit.sum(axis=0)).reshape(-1).tolist()
wrd2score = dict(zip(wrd_lst, score_lst))

df_tf_sel_3 = (pd.DataFrame()
             .from_dict(wrd2score, orient='index')
             .reset_index()
             .set_axis(['name', 'score'], axis=1)
             .sort_values(by='score', ascending=False)
            )

In [ ]:
print(f'No. of PubMed records: {df_101.shape}')
print(f'No. of unique UIs: {df_104['ui'].nunique()}')
print(f'No. of unique MeSH terms: {df_104['term'].nunique()}')

# No. of PubMed records: (180, 4)
# No. of unique UIs: 13
# No. of unique MeSH terms: 13

In [ ]:
set(df_84['term'].to_list()) & set(df_94['term'].to_list()) & set(df_104['term'].to_list())

In [ ]:
Cortical Spreading Depression[MH] AND 1966/01/01:1987/12/31[DP] AND medline[sb] AND hasabstract AND english[LA]

In [ ]:
# Common MeSH headings across all three B terms
set(df_84['term'].to_list()) & set(df_94['term'].to_list()) & set(df_104['term'].to_list())

Cortical Spreading Depression[MH] AND 1966/01/01:1987/12/31[DP] AND medline[sb] AND hasabstract AND english[LA]

'Calcium' 4
'Iodine Radioisotopes' 1
'Lithium' 5
'Magnesium' 0
'Manganese' 0
'Oxygen' 6
'Potassium' 4
'Sodium' 2
'Tritium' 0
'Xenon Radioisotopes' 10

tmp = pd.DataFrame({'s1': df_tf_sel_1['name'],
              's2': df_tf_sel_2['name'],
              's3': df_tf_sel_3['name']})

tmp.head()

In [ ]:
# Robust Rank Aggregation (Kolde et al., 2012)
# Partially based on R implementation
# TODO Drop pandas

import numpy as np
import pandas as pd

from scipy.stats import beta

def rank_matrix(glist, N=None, full=False):
    # Get the unique elements from the lists in glist
    u = pd.Index(set().union(*glist))
    
    # If N is None or empty, set it to the length of unique elements
    if N is None:
        N = len(u)
    
    if not full:
        # Initialize a matrix with 1s
        rmat = pd.DataFrame(
            np.ones((len(u), len(glist))),
            index=u,
            columns=range(len(glist))
        )
        # Expand N to match the number of columns in the matrix
        if isinstance(N, int):
            N = [N] * rmat.shape[1]
    else:
        # Initialize a matrix with NaNs
        rmat = pd.DataFrame(
            np.full((len(u), len(glist)), np.nan),
            index=u,
            columns=range(len(glist))
        )
        # Set N as the lengths of the individual lists in glist
        N = [len(g) for g in glist]
    
    # Fill in the rank values for each list in glist
    for i, g in enumerate(glist):
        rmat.loc[g, i] = [(j + 1) / N[i] for j in range(len(g))]
    
    return rmat


def beta_scores(r):
    # Count non-NA values
    n = np.sum(~np.isnan(r))
    
    # Create an array of ones of length n
    p = np.ones(n)
    
    # Sort r, placing NaNs at the end
    r = np.sort(r[~np.isnan(r)])
    
    # Compute the beta probabilities
    p = beta.cdf(r, a=np.arange(1, n + 1), b=n - np.arange(1, n + 1) + 1)
    
    return p

def correct_beta_pvalues(p, k):
    # Adjust the p-values and ensure they don't exceed 1
    p = min(p * k, 1)
    
    return p

def rho_scores(r):
    # Compute beta scores
    x = beta_scores(r)
    
    # Compute rho value
    rho = correct_beta_pvalues(np.nanmin(x), k=np.sum(~np.isnan(x)))
    
    return rho

In [ ]:
g_lst = [df_tf_sel_1['name'].to_list(),
         df_tf_sel_2['name'].to_list(),
         df_tf_sel_3['name'].to_list()]

rm_df = rank_matrix(g_lst)
# rm_df.transform(np.sort)

rm_df.apply(rho_scores, axis=1).sort_values()